### Testing New csv Load In Method

In [ ]:
# Must Pip Install python-decouple each time. 
#!pip install python-decouple

In [ ]:
# Testing a Progress Bar Install for the Vectorizer. 
#import pyprind
#import psutil

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
#import theano
import spacy
import s3
nlp = spacy.load('en')

In [2]:
# Everytime you enter
bucket = s3.Bucket('yelpchallenge1')

In [3]:
# Look inside the bucket.
bucket.contents

['datasets/',
 'datasets/df.csv',
 'notebooks/',
 'notebooks/data_cleanup.ipynb',
 'notebooks/vectorization_exploration.ipynb',
 'notebooks/yelp_data_initial_exploration.ipynb']

In [4]:
# Only have to run this once.
# Installs the .csv 'Locally' on SageMaker Instance

#bucket.get('datasets/df.csv', 'df.csv')

In [4]:
# Read df.csv
# Time to Load In: 8 Minutes.... 
df = pd.read_csv('df.csv')

/home/ec2-user/anaconda3/envs/ydc1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (0,1,2,3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
# check import
df.head()

,Unnamed: 0,stars,text,date,total_votes,tokens
0,0,1,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,7.0,"['total', 'horrible', 'service', 'crooks', 'ac..."
1,1,5,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,0.0,"['adore', 'travis', 'hard', 'rock', 'kelly', '..."
2,2,5,I have to say that this office really has it t...,2016-11-09 20:09:03,3.0,"['office', 'organized', 'friendly', 'phillipp'..."
3,3,5,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,0.0,"['went', 'lunch', 'steak', 'sandwich', 'delici..."
4,4,1,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,7.0,"['today', 'second', 'sessions', 'paid', 'sessi..."


In [6]:
# check datatypes of columns in dataframe
df.dtypes

Unnamed: 0      object
stars           object
text            object
date            object
total_votes    float64
tokens          object
dtype: object

In [7]:
# drop unwanted columns
df = df.drop(columns=['Unnamed: 0'])

In [8]:
df.head()

,stars,text,date,total_votes,tokens
0,1,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,7.0,"['total', 'horrible', 'service', 'crooks', 'ac..."
1,5,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,0.0,"['adore', 'travis', 'hard', 'rock', 'kelly', '..."
2,5,I have to say that this office really has it t...,2016-11-09 20:09:03,3.0,"['office', 'organized', 'friendly', 'phillipp'..."
3,5,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,0.0,"['went', 'lunch', 'steak', 'sandwich', 'delici..."
4,1,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,7.0,"['today', 'second', 'sessions', 'paid', 'sessi..."


In [9]:
df.dtypes

stars           object
text            object
date            object
total_votes    float64
tokens          object
dtype: object

In [10]:
df.shape

(6876898, 5)

In [11]:
# create mini-dataframe for testing
# want to make sure works locally on small dataset before scaling to entire dataset/AWS
#mini_df = df.head(10)

In [12]:
# exploring TFIDF Vectorizer

In [13]:
# new tokenizer
def tokenize(document):
    doc = nlp(document)
    return [token.lemma_ for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [14]:
# create variable to feed into TFIDF Vectorizer fit_transform
# to be updated to 'text' column of main dataframe (df['text']) for vectorization in AWS
data = df['text']

In [15]:
data.head()

0    Total bill for this horrible service? Over $8G...
1    I *adore* Travis at the Hard Rock's new Kelly ...
2    I have to say that this office really has it t...
3    Went in for a lunch. Steak sandwich was delici...
4    Today was my second out of three sessions I ha...
Name: text, dtype: object

In [16]:
data.shape

(6876898,)

In [17]:
#Checking for missing values/ Before Drop.nas
data.isna().sum()

131090

In [18]:
data.isna().any()

True

In [19]:
# Dropping na from Data
data = data.dropna()

In [20]:
#BEGONE NON-EXISTENT DATA
data.isna().sum()

0

In [21]:
data.isna().any()

False

In [22]:
data.shape

(6745808,)

In [23]:
data[2]

"I have to say that this office really has it together, they are so organized and friendly!  Dr. J. Phillipp is a great dentist, very friendly and professional.  The dental assistants that helped in my procedure were amazing, Jewel and Bailey helped me to feel comfortable!  I don't have dental insurance, but they have this insurance through their office you can purchase for $80 something a year and this gave me 25% off all of my dental work, plus they helped me get signed up for care credit which I knew nothing about before this visit!  I highly recommend this office for the nice synergy the whole office has!"

In [24]:
# vectorizer
tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))

# Create a vocabulary and get word counts per document
# Learn vocab and transform data into form we want

    # Started at 7:09 PM / 11-24-19
    # data_1 has 3,372,904 rows

# sparse = tfidf.fit_transform(data_1)

# Print word counts

# Get feature names to use as dataframe column headers

#dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame

#dtm.head(50)

In [26]:
# Attempting to Break Up Data
    # Breaking up data column into TWO Columns. 
        # data_1 & data_2 
        # data_1 will have 3,372,904 rows. 
        # data_2 will have 3,372,905 rows. 
#
print(data.shape)
data_1 = data[0:1686452]
data_2 = data[1686452]
print(data_1)

(6745808,)
0          Total bill for this horrible service? Over $8G...
1          I *adore* Travis at the Hard Rock's new Kelly ...
2          I have to say that this office really has it t...
3          Went in for a lunch. Steak sandwich was delici...
4          Today was my second out of three sessions I ha...
5          I'll be the first to admit that I was not exci...
6          Tracy dessert had a big name in Hong Kong and ...
7          This place has gone down hill.  Clearly they h...
8          I was really looking forward to visiting after...
9          It's a giant Best Buy with 66 registers.  I do...
10         Like walking back in time, every Saturday morn...
11         Walked in around 4 on a Friday afternoon, we s...
12         Wow. So surprised at the one and two star revi...
13         Michael from Red Carpet VIP is amazing ! I rea...
14         I cannot believe how things have changed in 3 ...
15         You can't really find anything wrong with this...
16         Gr

## Testing PyStemmer instead of NLTK

## Testing a Progress Bar install for Vectorizer

In [ ]:
# Testing a Progress Bar Install for the Vectorizer.
# Works pretty quickly..
# Given that nothing is running.. HA! 
# Also a pointless feature that I spent 20 minutes on. . . 

#import pyprind
#import psutil
#import random
#import time

#n = 100
#bar = pyprind.ProgBar(n, monitor=True)
#for i in range(n):
    #time.sleep(0.05)
#tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))
    #bar.update()
#print(bar)